In [21]:
# necessary Modules

import random
from hashlib import sha256, sha3_256
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.backends import default_backend

In [22]:
# define the safe prime p,q and generator g
def generate_dsa_parameters():
    # Generate DSA parameters with 1024-bit p and 160-bit q
    parameters = dsa.generate_parameters(key_size=1024, backend=default_backend())

    # Access the DSA parameters
    p = parameters.parameter_numbers().p
    q = parameters.parameter_numbers().q
    g = parameters.parameter_numbers().g

    print("Generated DSA Parameters:")
    print("p:", p)
    print("q:", q)
    print("g:", g)
    return p,q,g

# test cases
p,q,g = generate_dsa_parameters()

Generated DSA Parameters:
p: 125241971320815022083381328748826725447906443297474150296430219324427009259151421143036599868910898416514356335222372613905516377736232059446792302566937657853875215261333980424826735035280782174405630344461285766137517358369258028689295837185096461681201140006139269359471503308961761365095013606514938996129
q: 1295219081772726190133013508126708928666017963717
g: 65306889415282153490991345311956120180620400447610697550353209107196039939619706224815079862918708894148286291163557507290341374599954880093373676158826978624309132055850127181047729119857959219768351740174710157528150966365456280460423307831112926779952098084435048026151977248058534604451723981235290119930


In [23]:
# function to create s (secret key) and v (public key) with given g (generator) and p (prime mod)  
def generate_keys(g, p):
    # secret key s
    s = random.randint(1, p-2)
    # public key = g^s mod p
    v = pow(g, s, p)
    return s, v


# function to generate n random secret and public keys
def generate_n_random_keys(n):
    # a list to store all the pubkey and secrect keys
    keys_list = []
    for _ in range(n):
        # generate random keys
        x_temp, y_temp = generate_keys(g, p)
        # append random keys
        keys_list.append([x_temp, y_temp])
    return keys_list

# function to concat the real key to the random key list and shuffle
def concat_and_shuffle_total_keys(keys_list, x, y):
    # append the real key
    keys_list.append([x, y])
    # do the shuffle
    random.shuffle(keys_list)
    return keys_list

# function to get the real public key's corresponding position
def get_real_key_position(keys_list, y):
    # make the public keys into one list for index
    public_key_list = [key[1] for key in keys_list]
    # return the real public key index
    return public_key_list.index(y)

# function to get the Y, which is the string containing all the public key used in this signature
def get_Y(keys_list):
    # make the public keys into one list for getting the Y
    public_key_list = [key[1] for key in keys_list]
    # generate Y, which is the string containing all the public key used in this signature
    Y = ''.join([str(i) for i in public_key_list])
    return Y

# function to get the initial random value r and R used in the signature
def get_initial_random_r_and_R(g, p):
    r, R = generate_keys(g, p)
    return r, R

# function to generate the signature
def sign(msg, Y, g, r, p, keys_list, x_real, y_real):
    # find the real pubkey's position and +1 to get the next position
    idx = (get_real_key_position(total_keys_list, y_real) + 1) % len(total_keys_list)
    # initial the e_list with None (which represents c in the thesis)
    e_list = [None for _ in range(len(keys_list))]
    # initial the signature list with None 
    sig_list = [None for _ in range(len(keys_list))]
    # calculate the h with Y and using another hash function (thesis 4.1.1)
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16) % p
    h = pow(g, h, p)
    # calculate y wave (thesis 4.1.1)
    y_wave = pow(h, x_real, p)
    # calculate the first e with the random number "r" (thesis 4.1.2)
    e_initial = int(sha256((str(Y) + str(y_wave) +str(msg) + str(pow(g,r,p)) + str(pow(h,r,p))).encode()).hexdigest(), 16) % q
    # place the "e" into e_list
    e_list[idx] = e_initial
    # do the while loop, in order to calculate each position's value until the real public key's position
    while keys_list[idx][1] != y_real:
        # find the "e" used in last term loop
        e_last_one = e_list[idx]
        # generate the random value for the signature (thesis 4.1.3 start)
        _, random_sig = generate_keys(g, q)
        # place it to the position
        sig_list[idx] = random_sig
        # find this term cooresponding y
        temp_y = keys_list[idx][1]
        # do the calculation (thesis 4.1.3 for last two parts)
        # gy part
        calculation = (pow(g, random_sig, p)*pow(temp_y, e_last_one, p)) % p
        # hy part
        calculation_1 = (pow(h, random_sig, p)*pow(y_wave, e_last_one, p)) % p
        # calculate the e for this term (thesis 4.1.3 end)
        temp_e = int(sha256((str(Y) + str(y_wave) +str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16) % q
        # index += 1
        idx = (idx + 1) % len(total_keys_list)
        # place the "e"
        e_list[idx] = temp_e
        
    # q, see paper page 6, part 4, the first sentence. 
    # global q

    # do the real signature (thesis 4.1.4)
    s_real = (r - x_real * e_list[idx]) % (q)
    sig_list[idx] = s_real
    public_key_list = [key[1] for key in keys_list] 
    total_res = {
        'e1': e_list[0],
        'sig_list': sig_list,
        'public_key_list': public_key_list,
        'tag': y_wave,
        'msg': msg,
    }
    return total_res

# define the safe prime p q, and the g (primitive root of p) used in the whole process
p,q,g = generate_dsa_parameters()
msg = 'Hello, Ring Signature'
random_keys_count = 10

# main function
random_keys_list = generate_n_random_keys(random_keys_count)
x_real, y_real = generate_keys(g, p)
total_keys_list = concat_and_shuffle_total_keys(random_keys_list, x_real, y_real)
real_key_index = get_real_key_position(total_keys_list, y_real)
Y = get_Y(total_keys_list)
r, R = get_initial_random_r_and_R(g, q)
total_res = sign(msg, Y, g, r, p, total_keys_list, x_real, y_real)

display(total_res)

Generated DSA Parameters:
p: 126072118717677121749429922691186562502281398957613027561153372405839281428358090204163166317357820776082704143373061583449588187750151174128518815239276925018600059267959799719940957059601678453435133111935711499311823199937426299648129753571649424224189379660720991888356097138173863842301027007737835555619
q: 1192508907103966485609321235006755430379975715379
g: 65528222048170463637922188329924034806960655492322326160009945262876031145304705695367005175436623330075775246555052129144863130563794573588009182917489081750810784591349451206886306498944490877735166194177872893094958112808065998491983139896149795873645931828384961430670270176046712521577072709759868114083


{'e1': 770851936206709323915890067342743287745711783072,
 'sig_list': [807245298064038675000595013576697342595805770752,
  591728772196925528728918065442461124550686930904,
  552587549330022937534553340599117506904123801814,
  786109614688212606633680946136700065733315412807,
  408599457559899438675152865958469778790090539279,
  570606566477523749444720663076234620267299919366,
  1178134668036933867270306208852256683196451414273,
  1188459391583460561800067933694173916253769785418,
  1061141450681833248331339581017440943936200956963,
  447827004901752782979868411472943236771825056153,
  427414917852346426793715432839993084295420071793],
 'public_key_list': [112690032066307890585276283608443499555278627359106682926794139759466922943932866316711677823114825063315021337724507641960537783573474922384912991935463353302947185558429070664749588867201657794605273537556958383474018886232167654503935886006920353127352246944620654671981749781827357365410798208468102234568,
  264355982941261564897

In [24]:
# This part of code is used for checking the verification process.

def verify(total_res):
    public_key_list = total_res['public_key_list']
    Y = ''.join([str(i) for i in public_key_list])
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16) % p
    h = pow(g, h, p)

    sig_list = total_res['sig_list']
    temp_e_V = e1 = total_res['e1']
    msg = total_res['msg']
    tag = total_res['tag']
    
    for i in range(len(sig_list)):
        temp_sig = sig_list[i]
        temp_y = public_key_list[i]
        calculation = (pow(g, temp_sig, p)*pow(temp_y, temp_e_V, p)) % p
        calculation_1 = (pow(h, temp_sig, p)*pow(tag, temp_e_V, p)) % p
        temp_e_V = int(sha256((str(Y) + str(tag) + str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16) % q

    print('The verification result is:', temp_e_V == e1)
    return temp_e_V == e1



verification_res = verify(total_res)

The verification result is: True


In [25]:
# # This part of code is used for checking the linkability
# # if two signature's tag is identical, then they must come from the same signer

def check_tag(signature_res, signature_res_1):
    return signature_res['tag'] == signature_res_1['tag']

msg_1 = 'new test'
total_res_1 = sign(msg_1, Y, g, r, p, total_keys_list, x_real, y_real)
display(total_res_1)
print("total_res_1['tag'] == total_res['tag']:",check_tag(total_res, total_res_1))

{'e1': 1159925624835062360015507306943787943172119098618,
 'sig_list': [121943619669579895792437132606235789221160488440,
  570087822058946708851819171391077397068666158751,
  49166427057280022961178002349488499060408663029,
  433668717584996841183850772462762281308087143275,
  1180278428787718869923414397022897585573931294696,
  712693894611323273358565443041206428103341813608,
  678097451650768671345079130673505792221107398947,
  277735896836673094355779884350842036917483220721,
  811583236051240192594084229794792134025850405368,
  611126791978364014121595285611597498434699631711,
  807029829462672452655545563450362237236025543675],
 'public_key_list': [112690032066307890585276283608443499555278627359106682926794139759466922943932866316711677823114825063315021337724507641960537783573474922384912991935463353302947185558429070664749588867201657794605273537556958383474018886232167654503935886006920353127352246944620654671981749781827357365410798208468102234568,
  26435598294126156489746

total_res_1['tag'] == total_res['tag']: True
